In [1]:
from ollama_client import OllamaClient
from pydantic import BaseModel, Field
from tools import tavily_search
import datetime
import json
from openai_client import OpenAIClient as OpenAIClient
from deep_research_tools import deep_tavily_search
from IPython.display import display, Markdown

print(datetime.datetime.now().strftime("%d %b %Y"))

# loadenv
import dotenv
dotenv.load_dotenv()

provider = 'ollama'
if provider == "openai":
    Client  = OpenAIClient
    model1 = 'gpt-5-nano'
    model2 = 'gpt-5-mini'

else:
    Client = OllamaClient
    model1 = "qwen3:8b"
    model2 = "mistral-small3.2:latest"
    model2 = "magistral:latest"
    model2 = "qwen3:8b"
    model2 = "qwen2.5:14b"


28 Aug 2025


### Reformulate User Question

In [2]:
task = """you are an expert on simplifying user question for optimal google search. 
1. Identify 3 different  formulations of what the user is asking
2. keep the reformulations short and concise
3. the 3 formulations must be varied and must be useful and varied variations of the user query but must be strongly related to the user query
4. No need to write complete sentences but iinclude all important aspects"""

re_phrase_agent = Client(role = task, model_name=model2)

class RePhraseResult(BaseModel):
    formulation_1: str
    formulation_2: str
    formulation_3: str


re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?""", json_schema=RePhraseResult)

{'model': 'mistral-small3.2:latest', 'created_at': '2025-08-25T07:18:11.636543818Z', 'message': {'role': 'assistant', 'content': '{ "formulation_1": "improve Python code performance",\n  "formulation_2": "optimize Python code speed",\n  "formulation_3": "boost Python code efficiency" }'}, 'done_reason': 'stop', 'done': True, 'total_duration': 40038018335, 'load_duration': 6027615769, 'prompt_eval_count': 455, 'prompt_eval_duration': 26167258458, 'eval_count': 43, 'eval_duration': 7738606244}


{'text': '{ "formulation_1": "improve Python code performance",\n  "formulation_2": "optimize Python code speed",\n  "formulation_3": "boost Python code efficiency" }',
 'raw': {'model': 'mistral-small3.2:latest',
  'created_at': '2025-08-25T07:18:11.636543818Z',
  'message': {'role': 'assistant',
   'content': '{ "formulation_1": "improve Python code performance",\n  "formulation_2": "optimize Python code speed",\n  "formulation_3": "boost Python code efficiency" }'},
  'done_reason': 'stop',
  'done': True,
  'total_duration': 40038018335,
  'load_duration': 6027615769,
  'prompt_eval_count': 455,
  'prompt_eval_duration': 26167258458,
  'eval_count': 43,
  'eval_duration': 7738606244},
 'conversation_history': [{'role': 'user',
   'content': 'Hey Man. i love coding. \n                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?'},
  {'role': 'assistant',
   'content': '{ "formulation_1": "improve Python code perfo

In [8]:
print(re_phrase_agent.system_instructions)

You are a helpful Agent among a group of agents trying to solve a problem. Each agent is tasked with a part or the entire problem.
You will be given your task. You will have access to all the relevant information and tools. You can also see the work already done by other agents. Use that information if required.


## Instructions
1. View the context, the task executed, the results, the tool call results of the other agents.
2. Reason and execute your task.
3. You have access to multiple tools as listed. You can only call tools which are relevant to your task. Other agents might have executed other tools which you dont have access to.
4. Make the necessary tool calls if your tool descriptions match the task
5. your task will be enclosed in <YOUR TASK></YOUR TASK> tags. This is your task. Only execute this task.
6. The work done by other agents will be enclosed in <Agent: Agent Name></Agent: Agent Name> tags. There may be multiple of these.
7. Any information to be used as reference (fro

### Web Search Agent

In [14]:
task = """You are an expert on simplifying user question for optimal google search and then invoking search tool. 
1. Identify 3 different formulations of what the user is asking
2. keep the reformulations short and concise. 
3. Invoke the tavily_search if it will help provide an updated and more accurate answer to the question.
4. If invoking the tool, extract from the user question 3 different ways of formulating the same question and pass to the search tool.
5. If the question needs to refer to documentation, do a web search and answer
6. If the user asks for recent events or asks to refer to the web, then use this tool
7. Rephrase the user question in 3 ways and pass each of them to the tool
"""

re_phrase_agent = Client(role = task, model_name=model2, agent_name="SearchAgent")

class AnswerBasedOnContext(BaseModel):
    answer: str

search_queries = re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: What is the latest version of gpt out there?""", tools=[tavily_search])

search_queries

{'text': '',
 'raw': {'id': 'chatcmpl-C8LjT9UHKYpKvN2sFyz96PQDeIZTU',
  'object': 'chat.completion',
  'created': 1756106199,
  'model': 'gpt-5-mini-2025-08-07',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': None,
     'tool_calls': [{'id': 'call_vv5qycpRvDtdAcQAbuePUdOZ',
       'type': 'function',
       'function': {'name': 'tavily_search',
        'arguments': '{"query1":"What is the latest GPT model version released by OpenAI?","query2":"Which GPT model is the newest as of Aug 2025?","query3":"Latest version/name of GPT (GPT-4, GPT-4o, GPT-5) and its release date?","max_results":10}'}}],
     'refusal': None,
     'annotations': []},
    'finish_reason': 'tool_calls'}],
  'usage': {'prompt_tokens': 821,
   'completion_tokens': 471,
   'total_tokens': 1292,
   'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
   'completion_tokens_details': {'reasoning_tokens': 384,
    'audio_tokens': 0,
    'accepted_prediction_tokens': 0,
    're

In [4]:
import json
print(search_queries['tool_results']["tavily_search"])
print(len(search_queries['tool_results']["tavily_search"]))

<search_metadata>
Queries: ['What is the latest GPT version available today?', 'What is the most recent GPT model release?', 'What is the newest version of GPT in the market right now?']
Total Queries: 3
Max Results per Query: 15
Search Mode: Advanced
Raw Content: False
</search_metadata>

<query_section>
Query: What is the latest GPT version available today?
</query_section>

<chunk 1>
Query: What is the latest GPT version available today?
Title: What's new in Azure OpenAI in Azure AI Foundry Models?
URL: https://learn.microsoft.com/en-us/azure/ai-foundry/openai/whats-new
Score: 0.82186085
Published: N/A
Content: ### Evaluations guide ### Latest GPT-4o model available in the early access playground (preview) On August 6, 2024, OpenAI announced the latest version of their flagship GPT-4o model version `2024-08-06`. GPT-4o `2024-08-06` has all the capabilities of the previous version as well as: `2024-08-06` `2024-08-06` Azure customers can test out GPT-4o `2024-08-06` today in the new 

## Deep Research 1


#### Deep Research Clarification Asker

In [2]:
task = """You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
2. keep the questions short and concise.
3. The questions should clarify what the user wants to research / learn / understand / plan out.
4. Once the user answers the questions, use their responses to guide your research and provide more tailored information.

For example, if the user asks: 
Example 1:
User Question: Help me prepare an itinerary for my trip to Japan next month.
Clarifying questions: 
1. What specific cities in Japan are you planning to visit?
2. How many days do you have for your trip?
3. What type of activities are you interested in (e.g., cultural, outdoor, culinary)?
4. What is your budget for the trip?

Example 2:
User Question: Which city are you moving to?
Clarifying Questions:
1. Which city are you moving to?
2. What’s your reason for the move (e.g., job, studies, lifestyle change)?
3. What’s your estimated move-in date?
4. Do you need help finding housing, setting up utilities, understanding the local job market, or anything else?
5. What is your budget for the move and for monthly living expenses?


Example 3:
User Question:
I want to understand Autoencoders. Help me understand the key architectures, starting from basics to good models

Clarifying Questions:
1. What is your primary purpose for learning about autoencoders? (e.g., anomaly detection, dimensionality reduction, data generation, etc.)
2. What level of detail are you looking for in the architecture explanations? Do you want theoretical grounding, mathematical formulation, and implementation-level details (e.g., PyTorch or TensorFlow)?
3. Are you interested only in standard autoencoders or also in variants like variational autoencoders (VAEs), denoising autoencoders, sparse autoencoders, etc.?
4. What is your level of current understanding? (e.g., beginner, intermediate, advanced)
"""

clarifying_agent = Client(role = task, model_name=model1, agent_name="ClarifyingAgent")

class ClarifyingQuestions(BaseModel):
    question_1: str
    question_2: str
    question_3: str


#clarifying_questions = re_phrase_agent.invoke(query="""Help me plan a birthday party""", json_schema=ClarifyingQuestions)
clarifying_questions = clarifying_agent.invoke(query="""Help me understand where to buy good coffee beans""", 
                                               json_schema=ClarifyingQuestions)



clarifying_questions

{'model': 'qwen3:8b', 'created_at': '2025-08-27T20:11:10.712070126Z', 'message': {'role': 'assistant', 'content': '{"question_1": "Are you looking for specialty coffee beans or regular coffee beans?", "question_2": "Do you have a preferred roast level (light, medium, dark) or any specific flavor profile you\'re interested in?", "question_3": "Are you looking for organic, fair-trade, or ethically sourced coffee beans?"}\n'}, 'done_reason': 'stop', 'done': True, 'total_duration': 8528928103, 'load_duration': 2467489247, 'prompt_eval_count': 781, 'prompt_eval_duration': 1126024151, 'eval_count': 72, 'eval_duration': 4930364576}


{'text': '{"question_1": "Are you looking for specialty coffee beans or regular coffee beans?", "question_2": "Do you have a preferred roast level (light, medium, dark) or any specific flavor profile you\'re interested in?", "question_3": "Are you looking for organic, fair-trade, or ethically sourced coffee beans?"}\n',
 'raw': {'model': 'qwen3:8b',
  'created_at': '2025-08-27T20:11:10.712070126Z',
  'message': {'role': 'assistant',
   'content': '{"question_1": "Are you looking for specialty coffee beans or regular coffee beans?", "question_2": "Do you have a preferred roast level (light, medium, dark) or any specific flavor profile you\'re interested in?", "question_3": "Are you looking for organic, fair-trade, or ethically sourced coffee beans?"}\n'},
  'done_reason': 'stop',
  'done': True,
  'total_duration': 8528928103,
  'load_duration': 2467489247,
  'prompt_eval_count': 781,
  'prompt_eval_duration': 1126024151,
  'eval_count': 72,
  'eval_duration': 4930364576},
 'conversatio

{'question_1': 'What is your budget range for coffee beans per pound?',
 'question_2': 'Do you prefer single-origin or blended coffee beans?',
 'question_3': 'Are you looking for beans that are roasted recently or are you okay with beans that have been roasted a few weeks ago?'}

In [2]:
def deep_research_clarification_agent(deep_research_request, model, client):
    
    task = """You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

    1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
    2. keep the questions short and concise.
    3. The questions should clarify what the user wants to research / learn / understand / plan out.
    4. Once the user answers the questions, use their responses to guide your research and provide more tailored information.

    For example, if the user asks: 
    Example 1:
    User Question: Help me prepare an itinerary for my trip to Japan next month.
    Clarifying questions: 
    1. What specific cities in Japan are you planning to visit?
    2. How many days do you have for your trip?
    3. What type of activities are you interested in (e.g., cultural, outdoor, culinary)?
    4. What is your budget for the trip?

    Example 2:
    User Question: Which city are you moving to?
    Clarifying Questions:
    1. Which city are you moving to?
    2. What’s your reason for the move (e.g., job, studies, lifestyle change)?
    3. What’s your estimated move-in date?
    4. Do you need help finding housing, setting up utilities, understanding the local job market, or anything else?
    5. What is your budget for the move and for monthly living expenses?


    Example 3:
    User Question:
    I want to understand Autoencoders. Help me understand the key architectures, starting from basics to good models

    Clarifying Questions:
    1. What is your primary purpose for learning about autoencoders? (e.g., anomaly detection, dimensionality reduction, data generation, etc.)
    2. What level of detail are you looking for in the architecture explanations? Do you want theoretical grounding, mathematical formulation, and implementation-level details (e.g., PyTorch or TensorFlow)?
    3. Are you interested only in standard autoencoders or also in variants like variational autoencoders (VAEs), denoising autoencoders, sparse autoencoders, etc.?
    4. What is your level of current understanding? (e.g., beginner, intermediate, advanced)
    """

    clarifying_agent = client(role = task, model_name=model, agent_name="ClarifyingAgent")

    class ClarifyingQuestions(BaseModel):
        question_1: str
        question_2: str
        question_3: str


    #clarifying_questions = re_phrase_agent.invoke(query="""Help me plan a birthday party""", json_schema=ClarifyingQuestions)
    clarifying_questions = clarifying_agent.invoke(query=deep_research_request, 
                                                json_schema=ClarifyingQuestions)

    pre_r = json.loads(clarifying_questions['text'])
    r = "To help me answer your query, pleae answer the following questions:\n"

    for i in range(len(pre_r)):
        r += f"{i+1}. {pre_r[f'question_{i+1}']}\n"

    # Context till this node to be passed to other agents
    deep_research_agent_context = clarifying_agent.get_agent_context()
    return deep_research_agent_context, r

In [3]:
research_question  =  "Show me how to buy good coffee online"
deep_research_agent_context, r = deep_research_clarification_agent(research_question, model=model2, client=Client)
print(deep_research_agent_context)

{'model': 'qwen2.5:14b', 'created_at': '2025-08-27T20:47:39.15525443Z', 'message': {'role': 'assistant', 'content': '{ "question_1": "What type of coffee are you interested in buying (e.g., single-origin, blends, decaf)?", "question_2": "Are there specific brands or roasters you prefer?", "question_3": "How do you plan to brew your coffee at home?" }'}, 'done_reason': 'stop', 'done': True, 'total_duration': 12318574285, 'load_duration': 2910462638, 'prompt_eval_count': 826, 'prompt_eval_duration': 2168420925, 'eval_count': 64, 'eval_duration': 7220889577}
<Agent: ClarifyingAgent>
            ### Task to be done by the agent:
            You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

    1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
    2. keep the questions short and concise.
    3. The questions should clarify what the user wants to research / learn / und

In [4]:
def add_agent_response_and_user_answer(agent_response: str, user_answers: str) -> str:
    agent_response = """\n\nagent_response: """ + agent_response
    user_answers = """\n\nuser_answers: """ + user_answers

    return agent_response + user_answers


answer = """
        1. Whole Beans Arabica
        2. No
        3. V60
        """


next_context_to_be_passed = deep_research_agent_context + add_agent_response_and_user_answer(r, answer)

print(next_context_to_be_passed)

<Agent: ClarifyingAgent>
            ### Task to be done by the agent:
            You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

    1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
    2. keep the questions short and concise.
    3. The questions should clarify what the user wants to research / learn / understand / plan out.
    4. Once the user answers the questions, use their responses to guide your research and provide more tailored information.

    For example, if the user asks: 
    Example 1:
    User Question: Help me prepare an itinerary for my trip to Japan next month.
    Clarifying questions: 
    1. What specific cities in Japan are you planning to visit?
    2. How many days do you have for your trip?
    3. What type of activities are you interested in (e.g., cultural, outdoor, culinary)?
    4. What is your budget for the trip?

    Example 

In [5]:
Markdown(clarifying_agent.get_agent_context())

NameError: name 'clarifying_agent' is not defined

### Deep Research Search Agent

In [6]:
task = """You are an expert on observing Agent-User conversations and understanding user intent for making search tool calls. 
Observe the shared conversation between user and Agent and generate 6 questions which, when searched can answer the user's intent.
1. Identify 6 questions based on the conversation on which web search can be done.
2. keep the search questions short and concise. 
3. Invoke the deep_tavily_search if it will help provide an updated and more accurate answer to the question.
4. If invoking the tool, extract from the agent-user question 6 different ways of formulating the same question and pass to the search tool.
5. If the question needs to refer to documentation, do a web search and answer
6. If the user asks for recent events or asks to refer to the web, then use this tool
7. One of the 6 questions you make, must be about an expanded search space. For example, if the user wants to learn about Linear Regression, also include a search item for something like Lasso or Ridge which are related but still introduce some variation
8. The 6th question, as mentioned must go a bit beyond the exact search - something that might introduce a new perspective
9. Rephrase the user question in 6 ways and pass each of them to the tool
"""

deep_search_question_formulator_1 = Client(role = task, model_name=model2, agent_name="DeepSearchAgent1", history_from_other_agents = next_context_to_be_passed)
#deep_search_question_formulator_1.set_history_from_other_agents(history = next_context_to_be_passed)
class DeepSearchAgent1(BaseModel):
    question1: str
    question2: str
    question3: str
    question4: str
    question5: str
    question6: str

search_queries = deep_search_question_formulator_1.invoke(query=f"""Search the web with the rephrased 6 questions.""",
                                                           tools=[deep_tavily_search])
search_queries_full_context = deep_search_question_formulator_1.get_agent_context()
search_queries

{'model': 'qwen2.5:14b', 'created_at': '2025-08-27T20:49:27.778087923Z', 'message': {'role': 'assistant', 'content': 'คณะกรรมการทำความเข้าใจจากบทสนทนาของผู้ใช้และเอเจนต์ และสร้างคำถาม 6 ข้อที่ทำให้สามารถค้นหาคำตอบของเป้าหมายของผู้ใช้ได้\n1. จากบทสนทนาในเรื่องการซื้อโคมากาแฟออนไลน์ สร้างคำถาม 6 ข้อที่เหมาะสมในการค้นหาบนเว็บ\n2. ทำให้คำถามสั้นและกระชับ\n3. เรียกใช้งาน deep_tavily_search เพื่อค้นหาคำตอบที่ตรงความต้องการของผู้ใช้จากเว็บล่าสุด\n4. หากเรียกใช้เครื่องมือนี้ ให้สร้างคำถามอย่างน้อย 6 รูปแบบสำหรับการค้นหาจากคำถามเดียวเพื่อให้ได้ผลลัพธ์ที่ดีที่สุด\n5. หาคำถามที่ต้องอ้างอิงเอกสาร จากเว็บไซต์และตอบผู้ใช้\n6. หาว่าผู้ใช้ถามเกี่ยวกับเหตุการณ์ล่าสุดหรือขอให้ค้นจากเว็บ แล้วใช้เครื่องมือนี้\n7. หนึ่งในคำถามทั้งหมด ควรขยายพื้นที่ในการค้นหา เช่น ถ้าผู้ใช้ต้องการเรียนรู้เกี่ยวกับ Regression แบบเส้นตรง เสนอข้อมูลเกี่ยวกับ Lasso หรือ Ridge ซึ่งเป็นสิ่งที่เกี่ยวข้องแต่มีความแตกต่าง\n8. คำถามที่ 6 ควรให้มุมมองที่หลากหลาย\n\nจึงสร้างคำถามในการค้นหาดังนี้:\n\n', 'tool_calls': [{'function': {'nam

{'text': 'คณะกรรมการทำความเข้าใจจากบทสนทนาของผู้ใช้และเอเจนต์ และสร้างคำถาม 6 ข้อที่ทำให้สามารถค้นหาคำตอบของเป้าหมายของผู้ใช้ได้\n1. จากบทสนทนาในเรื่องการซื้อโคมากาแฟออนไลน์ สร้างคำถาม 6 ข้อที่เหมาะสมในการค้นหาบนเว็บ\n2. ทำให้คำถามสั้นและกระชับ\n3. เรียกใช้งาน deep_tavily_search เพื่อค้นหาคำตอบที่ตรงความต้องการของผู้ใช้จากเว็บล่าสุด\n4. หากเรียกใช้เครื่องมือนี้ ให้สร้างคำถามอย่างน้อย 6 รูปแบบสำหรับการค้นหาจากคำถามเดียวเพื่อให้ได้ผลลัพธ์ที่ดีที่สุด\n5. หาคำถามที่ต้องอ้างอิงเอกสาร จากเว็บไซต์และตอบผู้ใช้\n6. หาว่าผู้ใช้ถามเกี่ยวกับเหตุการณ์ล่าสุดหรือขอให้ค้นจากเว็บ แล้วใช้เครื่องมือนี้\n7. หนึ่งในคำถามทั้งหมด ควรขยายพื้นที่ในการค้นหา เช่น ถ้าผู้ใช้ต้องการเรียนรู้เกี่ยวกับ Regression แบบเส้นตรง เสนอข้อมูลเกี่ยวกับ Lasso หรือ Ridge ซึ่งเป็นสิ่งที่เกี่ยวข้องแต่มีความแตกต่าง\n8. คำถามที่ 6 ควรให้มุมมองที่หลากหลาย\n\nจึงสร้างคำถามในการค้นหาดังนี้:\n\n',
 'raw': {'model': 'qwen2.5:14b',
  'created_at': '2025-08-27T20:49:27.778087923Z',
  'message': {'role': 'assistant',
   'content': 'คณะกรรมก

In [7]:
def deep_search_search_agent_1(model=model1):
    task = """You are an expert on observing Agent-User conversations and understanding user intent for making search tool calls. 
    Observe the shared conversation between user and Agent and generate 6 questions which, when searched can answer the user's intent.
    1. Identify 6 questions based on the conversation on which web search can be done.
    2. keep the search questions short and concise. 
    3. Invoke the deep_tavily_search if it will help provide an updated and more accurate answer to the question.
    4. If invoking the tool, extract from the agent-user question 6 different ways of formulating the same question and pass to the search tool.
    5. If the question needs to refer to documentation, do a web search and answer
    6. If the user asks for recent events or asks to refer to the web, then use this tool
    7. One of the 6 questions you make, must be about an expanded search space. For example, if the user wants to learn about Linear Regression, also include a search item for something like Lasso or Ridge which are related but still introduce some variation
    8. The 6th question, as mentioned must go a bit beyond the exact search - something that might introduce a new perspective
    9. Rephrase the user question in 6 ways and pass each of them to the tool
    """

    deep_search_question_formulator_1 = Client(role = task, model_name=model, agent_name="DeepSearchAgent1", history_from_other_agents = next_context_to_be_passed)
    #deep_search_question_formulator_1.set_history_from_other_agents(history = next_context_to_be_passed)
    class DeepSearchAgent1(BaseModel):
        question1: str
        question2: str
        question3: str
        question4: str
        question5: str
        question6: str

    search_queries = deep_search_question_formulator_1.invoke(query=f"""Search the web with the rephrased 6 questions.""",
                                                            tools=[deep_tavily_search])
    search_queries_full_context = deep_search_question_formulator_1.get_agent_context()
    return search_queries_full_context, search_queries

In [8]:
search_queries_full_context, search_queries = deep_search_search_agent_1(model=model1)

{'model': 'qwen3:8b', 'created_at': '2025-08-27T20:50:02.133608121Z', 'message': {'role': 'assistant', 'content': '<think>\nOkay, let\'s tackle this. The user wants to know how to buy good coffee online. The agent asked clarifying questions, and the user answered they want whole Arabica beans, no specific brands, and they use a V60 brewer. \n\nFirst, I need to generate 6 search questions based on this info. The main topics are Arabica whole beans, V60 brewing, and no brand preferences. \n\nThe first five questions should be variations of the user\'s query. For example, "Best Arabica coffee beans for V60 brewing" and "How to select quality whole coffee beans for V60." Then, the sixth should expand a bit, maybe include something about brewing techniques or pairing. \n\nWait, the sixth needs to go beyond the exact search. Maybe include something about coffee roasting or storage? Or how different roasts affect V60 performance. That adds a new perspective. \n\nLet me check the tools. The de

In [9]:
search_queries['tool_results']

{'deep_tavily_search': '<search_metadata>\nQueries: [\'Best Arabica coffee beans for V60 brewing\', \'How to select quality whole coffee beans for V60\', \'Arabica coffee beans suitable for pour-over methods\', \'Top-rated coffee beans for V60 brewers 2025\', \'What factors to consider when buying Arabica whole beans for V60\', \'How does coffee roast level affect V60 brewing performance\']\nTotal Queries: 6\nMax Results per Query: 15\nSearch Mode: Advanced\nRaw Content: False\n</search_metadata>\n\n<query_section>\nQuery: Best Arabica coffee beans for V60 brewing\n</query_section>\n\n<chunk 1>\nQuery: Best Arabica coffee beans for V60 brewing\nTitle: Just started with pour over coffee! Suggestion for ...\nURL: https://www.reddit.com/r/pourover/comments/1egkpn7/just_started_with_pour_over_coffee_suggestion_for/\nScore: 0.98587\nPublished: N/A\nContent: I would suggest onyx. I bought a sampler from them & i was blown away, It literally felt like diving into a new world of coffee flavors

### Final Research Report Creator

In [10]:
task = """You are a Research expert who produces expert level research reports by observign questions and context. 
The research report must answer the user question enclosed in <research_question> tags. The search results are enclosed in <search_results> tags.
Follow the following guidelines while preparing the report:
1. First, try really hard to understand the intent of the research question, and look at the web search results. 
2. Make a plan on how to organize the information to properly answer the user question
3. The report must be in markdown format with proper headings, tables, etc.
4. The report must start with a 3 - 10 line direct answer to the user question with a heading: Summary.
5. Proceed to then present the detailed research on the user topic. Use information present only in the search results. 
6. Summarize the overall thoughts in a table at the end of the report in markdown format.

<research_question>
{research_question}
</research_question>


<search_results>
{search_results}
</search_results>

""".format(research_question  =  research_question, search_results = search_queries['tool_results']['deep_tavily_search'])

deep_search_report_generator = Client(role = task, model_name=model2, agent_name="DeepResearchReportAgent1")


deep_search_report_generator_results = deep_search_report_generator.invoke(query=task)
deep_search_report_generator_full_context = deep_search_report_generator.get_agent_context()


{'model': 'qwen2.5:14b', 'created_at': '2025-08-27T20:53:46.084781866Z', 'message': {'role': 'assistant', 'content': 'When considering how coffee roast level affects V60 brewing performance, several key factors come into play:\n\n1. **Complexity and Flavor Profiles**:\n   - Light to medium roasted coffees generally have cleaner, brighter levels of acidity and lighter body and texture.\n   - These roasts are ideal for wet-processed (washed) single origin coffees that highlight citrus and floral notes.\n   \n2. **Extraction Efficiency**:\n   - Lighter roasts take longer to extract due to their higher density, creating more fines during grinding which can slow down water flow through the coffee bed.\n   - This gives you more control over the brewing process but may result in a slower drain rate compared to darker roasts.\n\n3. **Solubility and Brewing Time**:\n   - Lighter roasted coffees often require a finer grind than darker ones for optimal extraction.\n   - Brew times will typically 

In [12]:
len(task)

79404

In [13]:
Markdown(deep_search_report_generator_results['text'])

When considering how coffee roast level affects V60 brewing performance, several key factors come into play:

1. **Complexity and Flavor Profiles**:
   - Light to medium roasted coffees generally have cleaner, brighter levels of acidity and lighter body and texture.
   - These roasts are ideal for wet-processed (washed) single origin coffees that highlight citrus and floral notes.
   
2. **Extraction Efficiency**:
   - Lighter roasts take longer to extract due to their higher density, creating more fines during grinding which can slow down water flow through the coffee bed.
   - This gives you more control over the brewing process but may result in a slower drain rate compared to darker roasts.

3. **Solubility and Brewing Time**:
   - Lighter roasted coffees often require a finer grind than darker ones for optimal extraction.
   - Brew times will typically be longer with lighter roasts due to their solubility properties.
   
4. **Chemical Composition**:
   - Different compounds in coffee, like organic acids and simple sugars, extract at different rates.
   - Lighter roasted coffees tend to release more delicate flavors during the early stages of brewing, making them ideal for showcasing nuanced flavor profiles.

5. **Water Temperature Considerations**:
   - Water temperatures between 195°-205°F (85°C-96°C) are recommended.
   - Higher water temperatures can easily extract bitter compounds from darker roasts, while cooler temperatures might under-extract lighter roasts, resulting in sour or weak flavors.

6. **Bloom and Drain Time**:
   - Lighter roasted coffees often produce a bigger bloom (carbon dioxide release during the first few seconds of pouring) compared to darker ones.
   - Adjusting the grind size and water temperature can help manage this effect, ensuring even extraction and flavor balance.

In summary, lighter roasts are generally more suitable for V60 brewing due to their ability to showcase complex flavors and nuances. However, the optimal roast level can vary based on personal preference and coffee origin characteristics. Experimentation with different grind sizes and water temperatures is key to achieving the best results in a V60 setup.

### Recommendations:
- **For Light Roasts**: Use finer grind settings, aim for longer brew times (1:30 - 2:30 minutes), and maintain water temperature around 205°F.
- **For Medium Roasts**: Opt for slightly coarser grinds, moderate brewing time (1:45 - 2:15 minutes), and keep temperatures between 198° - 202°F.
- **For Darker Roasts**: Employ coarsest grind possible to avoid over-extraction, shorter brew times (under 2 minutes) and cooler water around 195°F.

By understanding these nuances, you can fine-tune your V60 brewing technique to bring out the best in any coffee roast level.

In [55]:
search_queries['tool_results']['deep_tavily_search']

'<search_metadata>\nQueries: [\'Best ways to buy Blue Tokai whole bean coffee online under 1000 INR\', \'Where to purchase authentic Blue Tokai coffee beans in India within 1000 INR budget\', \'Online retailers offering high-quality Blue Tokai coffee beans under 1000 INR\', \'How to verify authenticity of Blue Tokai coffee beans purchased online\', \'Top websites to buy Blue Tokai coffee beans in India with price comparisons\', \'Sustainable and ethical ways to purchase Blue Tokai coffee beans online within 1000 INR\']\nTotal Queries: 6\nMax Results per Query: 15\nSearch Mode: Advanced\nRaw Content: False\n</search_metadata>\n\n<query_section>\nQuery: Best ways to buy Blue Tokai whole bean coffee online under 1000 INR\n</query_section>\n\n<chunk 1>\nQuery: Best ways to buy Blue Tokai whole bean coffee online under 1000 INR\nTitle: Buy Blue Tokai Coffee Online - Distacart\nURL: https://www.distacart.com/collections/blue-tokai-coffee?srsltid=AfmBOorbeoiFp_croS0EeXjrlHHWlgpEnOSUnSHOj5tk3_

### Generic Tool calling function

In [ ]:
def call_tools():
    

### Agent that summarizes the Answer

In [6]:
task = """You are an Expert on Answering a User Question based on provided context. 
Study the context deeply and answer the question asked by the user. Use a table in the end to summarize the answer"""
model1="qwen3:1.7b"
model2="llama3.2-vision:11b"
model3="gemma3:4b"


re_phrase_agent = OllamaClient(role = task, model_name=model2)

class AnswerBasedOnContext(BaseModel):
    answer: str
    


re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?""", json_schema=RePhraseResult)

HTTPError: 404 Client Error: Not Found for url: http://localhost:11434/api/chat